*This is an interactive guide walking you through basics of SAQL, up to most advanced and powerful possibilities of Wave Analytics.*

[< Back to homepage](/notebooks)

# Time Based Analysis

Time-based analysis relies heavily on date functions and ranges, which are documented [here](https://developer.salesforce.com/docs/atlas.en-us.bi_dev_guide_eql.meta/bi_dev_guide_eql/bi_eql_functions_date.htm). Some common examples are detailed below.

## MTD (Month To Date)
Calculate the total value of the opportunities expected to close from the beginning of the month through today.

In [23]:
%%saql
q = load "opportunities";
q = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current month".."current day"];
q = group q by all;
q = foreach q generate sum('Amount') as 'sum_Amount';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_Amount
0,21078458.0


The relative date range filters for the beginning of the month through today. As the month or day changes, these keywords will refer to the current month and day.

## YTD (Year To Date)

Calculate the total value of the opportunities expected to close from the beginning of the year through today.

In [24]:
%%saql
q = load "opportunities";
q = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current year".."current day"];
q = group q by all;
q = foreach q generate sum('Amount') as 'sum_Amount';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_Amount
0,178020414.0


## QTD (Quarter To Date)

Calculate the total value of the opportunities expected to close from the beginning of the quarter through today.

In [25]:
%%saql
q = load "opportunities";
q = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current quarter".."current day"];
q = group q by all;
q = foreach q generate sum('Amount') as 'sum_Amount';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_Amount
0,21078458.0


## QoQ (Quarter over Quarter)
Calculate the percent change in total value of the opportunities expected to close for the current quarter compared to last quarter.

In [26]:
%%saql
q = load "opportunities";
curr = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current quarter".."current quarter"];
prev = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["1 quarter ago".."1 quarter ago"];
QoQ = group curr by all, prev by all;
QoQ = foreach QoQ generate (sum(curr['Amount'])/sum(prev['Amount'])) - 1 as 'sum_QoQ';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_QoQ
0,-0.152189


** Query Explanation:**

After loading in the opportunity dataset, we split it into two streams. One stream, curr, gets data for the current quarter using the date range `["current quarter".."current quarter"]` which filters for data ranging from the start of the current quarter to the end of the current quarter. The other stream, prev, gets data for the previous quarter. We then bring the two streams back together using a co-group. Since we are trying to calculate the overall QoQ number, we can group by alland then perform the QoQ calculation.

### Windowing Alternative

Let us see how we can do the same QoQ using Windowing.

In [1]:
%%saql
q = load "opportunities";
q = group q by ('CloseDate_Year', 'CloseDate_Quarter');
q = foreach q generate 'CloseDate_Year' + "~~~" + 'CloseDate_Quarter' as 'CloseDate_Year~~~CloseDate_Quarter',
    ((sum('Amount') / sum(sum('Amount')) over([-1 .. -1] partition by all order by ('CloseDate_Year', 'CloseDate_Quarter'))) - 1) as 'sum_QoQ';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,CloseDate_Year~~~CloseDate_Quarter,sum_QoQ
0,2015~~~1,NaN
1,2015~~~2,4.060994
2,2015~~~3,-0.128742
3,2015~~~4,1.249819
4,2016~~~1,0.140070
5,2016~~~2,0.046936
6,2016~~~3,-0.491370
7,2016~~~4,-0.304678


After loading the opportunity dataset, We are projecting Year and Quarter as we wanted to see the result in a Time Series and the formula we are using here is (Current Quarter/Previous Quarter) - 1. So if we have a close look at this particular line

```pig
((sum('Amount') / sum(sum('Amount')) over([-1 .. -1] partition by all order by ('CloseDate_Year', 'CloseDate_Quarter'))) - 1)
```

The order of execution will be to order the dataset by Year and Quarter ascending and the sum('Amount') which will give the Sum for the current quarter and sum(sum('Amount')) which will give us the sum for the previous quarter because over(-1 .. -1) indicates that we go one previous quarter and subtracting -1 at the end to get QoQ. Since the partitioning is by “all”, windowing will happen across all Year-Quarter.

## QTD over QTD

Calculate the percent change in total value of the opportunities expected to close for the current quarter to date compared to last quarter to date.

In [8]:
%%saql
q = load "opportunities";
curr = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day')
    in ["current quarter".."current day"];
prev = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day')
    in ["1 quarter ago".."current day - 1 quarter"];
QoQ = group curr by all, prev by all;
QoQ = foreach QoQ generate (sum(curr['Amount'])/sum(prev['Amount'])) - 1 as 'sum_QoQ';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_QoQ
0,-0.035968


**Query Explanation:** This query utilizes techniques from the QTD and QoQ calculations. Similar to the QoQ query, we split the dataset into curr and prev streams; however, rather than filtering for the entire quarter, we filter for the quarter to date as we did in the QTD query. The way to get the same number of days from the previous quarter as the number of days elapsed in the current quarter is by using date math. `"current day - 1 quarter"` returns the current day relative to the previous quarter.

## Current QTD over last year same QTD

Calculate the percent change in total value of the opportunities expected to close for the current quarter to date compared to the same period last year.

In [28]:
%%saql
q = load "opportunities";
curr = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current quarter".."current day"];
prev = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current quarter - 1 year".."current day - 1 year"];
QoQ = group curr by all, prev by all;
QoQ = foreach QoQ generate (sum(curr['Amount'])/sum(prev['Amount'])) - 1 as 'sum_QoQ';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_QoQ
0,1.774138


**Query Explanation:** This query is similar to the QTD over QTD calculation. The only difference is that we refer to the previous year using relative date math `["current quarter - 1 year".."current day - 1 year"]`.

## Most recent available month

There are various reasons that your data could be out of date. For example, your bookings from an external data source may only get updated once a month on the 5th. On March 1 - 4, you still want to report off of January data. In these cases, standard relative dates may not be appropriate. Filtering for “previous month” on February 28th will give us our previous month’s bookings data; however, on March 1st, the “previous month” filter will look for data for February’s bookings data, which does not get entered into the system until four more days. The work around for this is to use the following steps:

In [23]:
%%saql
q = load "opportunities";
q = group q by ('CloseDate_Year', 'CloseDate_Month');
q = foreach q generate 'CloseDate_Year', 'CloseDate_Month', sum('Amount') as 'sum_Amount';
q = order q by ('CloseDate_Year' desc, 'CloseDate_Month' desc);
q = limit q 1;

User "kim@waverocks.de" running query on https://na30.salesforce.com


,CloseDate_Month,CloseDate_Year,sum_Amount
0,12,2016,4571092


**Query Explanation:** The objective of this query is to order the results by date in descending order and then select the first record to get the most recent date. This is achieved by ordering by the year and then month, both in descending order. The sequence in which you order is important here; for example, ordering by month and then year would give you different results. Since we are trying to find the most recent month available, we apply a group at the
year-month level. If we wanted the most recent available day, the grouping would be `('CloseDate_year', 'CloseDate_month', 'CloseDate_day')`. For most recent available quarter, `('CloseDate_year', 'CloseDate_quarter')`. If we were to try using standard relative dates like in the query below, the result would be $0 because there is no data for the current month.

In [17]:
%%saql
q = load "opportunities";
q = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current quarter".."current quarter"];
q = group q by all;
q = foreach q generate sum('Amount') as 'sum_Amount';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_Amount
0,177416756.0


Please note that $0 is not the incorrect result for the query. Rather, the query itself does not address the question that is proposed.

## MoM on most recent available month

Calculate the percent change in total value of the opportunities expected to close for the most
recent available month compared to the month before it.

In [19]:
%%saql
q = load "opportunities";
q = group q by ('CloseDate_Year', 'CloseDate_Month');
q = foreach q generate 'CloseDate_Year', 'CloseDate_Month', sum('Amount') as 'sum_Amount';
q = order q by ('CloseDate_Year' desc, 'CloseDate_Month' desc);
curr = limit q 1;
prev = offset q 1;
prev = limit prev 1;
MoM = group curr by all, prev by all;
MoM = foreach MoM generate (sum(curr['sum_Amount'])/sum(prev['sum_Amount'])) - 1 as 'sum_MoM';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_MoM
0,-0.873369


**Query Explanation:** This query builds upon the query to get the most recent available month. After ordering by year and month, we create two separate streams. currrepresents the most recent available month and prevrepresents the preceding month. curris the first record in the projection which is why we apply the limit of 1. We can get prev, which is the second record in the projection, by offsetting the projection by 1 and then getting the first record of this projection.

## Windowing Alternative

Let us see how we can do the same MoM on recent available month using Windowing.

In [21]:
%%saql
q = load "opportunities";
q = group q by ('CloseDate_Year', 'CloseDate_Month');
q = foreach q generate 'CloseDate_Year', 'CloseDate_Month',
    ((sum('Amount') / sum(sum('Amount')) over([1 .. 1] partition by all order by ('CloseDate_Year' desc, 'CloseDate_Month' desc))) - 1) * 100 as 'sum_MostRecentAvailableMoM';
q = order q by ('CloseDate_Year', 'CloseDate_Month');

User "kim@waverocks.de" running query on https://na30.salesforce.com


,CloseDate_Month,CloseDate_Year,sum_MostRecentAvailableMoM
0,03,2015,NaN
1,04,2015,35.782624
2,05,2015,12.635092
3,06,2015,42.133855
4,07,2015,-34.095057
5,08,2015,-27.230394
6,09,2015,85.540045
7,10,2015,42.240955
8,11,2015,45.666697
9,12,2015,-21.121832


**Query Explanation:**
```pig
((sum('Amount') / sum(sum('Amount')) over([1 .. 1] partition by all order by ('CloseDate_Year' desc, 'CloseDate_Month' desc))) - 1)
```

The order of execution will be to order the dataset by Year and Month ascending and the `sum('Amount')` which will give the Sum for the current month and `sum(sum('Amount'))` which will give us the sum for the previous month because `over([1 .. 1])` indicates that we go one month previous and subtracting -1 at the end to get MoM. Since the partitioning is by “all”, windowing will happen across all Year-Month. At the end since we are limiting the result set
by 1, we will only get the Most recent available Month in the dataset.

Note that even though the windowing function contains the keywords “order by”, the resulting stream is not guaranteed to be ordered (the “order by” here is for the calculation). This is why there is a separate “order by” statement following the windowing function.

## Timeline Comparisons

Another type of time-based analysis is comparing how we are trending in the current quarter versus the same time last year. This type of analysis is useful for seeing if we are on-track, lagging, or ahead by using a previous time period as a benchmark.

In [40]:
%%saql
q = load "opportunities";
ThisQ = filter q by date('CreatedDate_Year', 'CreatedDate_Month', 'CreatedDate_Day') in ["current quarter".."current quarter"];
LastQ = filter q by date('CreatedDate_Year', 'CreatedDate_Month', 'CreatedDate_Day') in ["current quarter - 1 year".."current quarter - 1 year"];
LastQGrp = cogroup ThisQ by (CreatedDate_Week) left, LastQ by (CreatedDate_Week);
LastQGrp = foreach LastQGrp generate
    first(ThisQ['CreatedDate_Year']) + "~~~" + ThisQ['CreatedDate_Week'] as 'CreatedDate_Year~~~CreatedDate_Week',
    "1 Year Ago" as 'Date', sum(LastQ['Amount']) as 'sum_Amount';
ThisQ = group ThisQ by (CreatedDate_Year, CreatedDate_Week);
ThisQ = foreach ThisQ generate 'CreatedDate_Year' + "~~~" + 'CreatedDate_Week' as 'CreatedDate_Year~~~CreatedDate_Week',
    "This Quarter" as 'Date',
    sum('Amount') as 'sum_Amount';
c = union ThisQ, LastQGrp;

User "kim@waverocks.de" running query on https://na30.salesforce.com


,CreatedDate_Year~~~CreatedDate_Week,Date,sum_Amount
0,2016~~~14,This Quarter,19153027.0
1,2016~~~15,This Quarter,19110473.0
2,2016~~~16,This Quarter,10769424.0
3,2016~~~17,This Quarter,10283451.0
4,2016~~~18,This Quarter,14849622.0
5,2016~~~19,This Quarter,10523830.0
6,2016~~~20,This Quarter,11574725.0
7,2016~~~21,This Quarter,4580310.0
8,2016~~~14,1 Year Ago,5374352.0
9,2016~~~15,1 Year Ago,9086455.0


Dashboard JSON version:

```json
"pigql": "q = load \"opportunities\"; ThisQ = filter q by date('CreatedDate_Year', 'CreatedDate_Month', 'CreatedDate_Day') in [\"current quarter\"..\"current quarter\"]; LastQ = filter q by date('CreatedDate_Year', 'CreatedDate_Month', 'CreatedDate_Day') in [\"current quarter - 1 year\"..\"current quarter - 1 year\"]; LastQGrp = cogroup ThisQ by (CreatedDate_Week) left, LastQ by (CreatedDate_Week); LastQGrp = foreach LastQGrp generate first(ThisQ['CreatedDate_Year']) + \"~~~\" + ThisQ['CreatedDate_Week'] as 'CreatedDate_Year~~~CreatedDate_Week', \"1 Year Ago\" as 'Date', sum(LastQ['Amount']) as 'sum_Amount'; ThisQ = group ThisQ by (CreatedDate_Year, CreatedDate_Week); ThisQ = foreach ThisQ generate 'CreatedDate_Year' + \"~~~\" + 'CreatedDate_Week' as 'CreatedDate_Year~~~CreatedDate_Week', \"This Quarter\" as 'Date', sum('Amount') as 'sum_Amount'; c = union ThisQ, LastQGrp;",
```

In this example we have two SAQL streams. One for this quarter's data which should hopefully be straightforward and one for the same quarter last year. To explain what we did with the 1 Year Ago stream, it's important to note that in order to use the time chart the final SAQL stream must be grouped by a valid date dimension and include the year and if we need to include the year we need to make sure the two separate data streams have the same year.

To manipulate the year in last year data stream, we cogrouped this quarter with 1 year ago using the CreatedDate_Week dimension. In the projection, we took the year (we needed to use the first() function to get this since it wasn't used in my grouping) and month from the "This Quarter" stream and the sum of Amount from the "1 Year Ago" stream. This projection gives the count of created cases by week for the quarter one year ago but instead of my date field containing 2014 it contains 2015 so the data will show up on the time chart.

Finally we union the two streams together which are both grouped by year and month.